In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"
import os
import regex
import matplotlib.pyplot as plt
import random

In [15]:
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
from wordcloud import WordCloud
from nltk.corpus import stopwords, words
from nltk.tokenize import WordPunctTokenizer, word_tokenize
from string import punctuation
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sandeepkumaryedla/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sandeepkumaryedla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sandeepkumaryedla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
# Define the path to the data directory
PATH = "Data"

# Get a list of all folders (directories) in the data directory
folders = os.listdir(PATH)

# Initialize an empty list to store data
data = []

# Loop through each folder
for folder in folders:
    # Create the full path to the folder
    folder_path = os.path.join(PATH, folder)
    
    # Skip non-directory files
    if not os.path.isdir(folder_path):
        continue
    
    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    
    # Loop through each file
    for file in files:
        try:
            # Open the file and read its contents
            with open(os.path.join(folder_path, file)) as f:
                contents = " ".join(f.readlines())
                
                # Extract the file name, folder name, and contents, and append them to the data list
                data.append([file.split(".")[0], folder, contents])
        except Exception as e:
            pass

# Create a DataFrame from the data list with column names 'ID', 'Category', and 'Content'
df = pd.DataFrame(data, columns=['ID', 'Category', 'Content'])

# Save the DataFrame to a CSV file named 'combine.csv' without index
df.to_csv("combine.csv", index=False)


In [17]:

# Read data from a CSV file into a Pandas DataFrame
df = pd.read_csv("combine.csv")

# Generate a bar plot of the value counts of the 'Category' column in the DataFrame
df['Category'].value_counts().plot.bar()

# Load English stopwords from the NLTK library
en_stopwords = stopwords.words('english')

# Initialize a WordNetLemmatizer object for lemmatization
word_lemmatizer = WordNetLemmatizer()

# Define a function to clean contents with stopwords
def clean_contents_w_stopwords(contents):
    # Tokenize the input string into words
    tokenized_word = word_tokenize(str(contents).replace("\n", "").replace("\\", "").replace(">", "").strip())
    tokenized_word_w_stopwords = []
    for word in tokenized_word:
        # Remove non-Latin characters from each word using regex
        word = regex.sub(u'\p{^Latin}', u'', word.lower())
        # Filter out words that are shorter than 4 characters and present in the English stopword list
        if len(word) > 3 and word.strip().lower() not in en_stopwords:
            # Lemmatize each word using WordNetLemmatizer
            word = word_lemmatizer.lemmatize(word)
            tokenized_word_w_stopwords.append(word)
    # Join the filtered and lemmatized words back into a single string separated by spaces
    tokenized_word_w_stopwords = " ".join(tokenized_word_w_stopwords)
    return tokenized_word_w_stopwords

# Extract the 'Content' column of the row in the DataFrame where the 'ID' column has a value of 14991,
# and store it as a string in the 'comment_words' variable
comment_words = df[df['ID'] == 14991]['Content'].to_string().replace("\n", "").replace("\\", "").replace(">", "").strip()

# Print the original 'comment_words' string
print(comment_words)

# Call the 'clean_contents_w_stopwords()' function on the 'comment_words' string
# to tokenize, clean, and lemmatize the words in the string
tokenized_word_w_stopwords = clean_contents_w_stopwords(comment_words)

# Print the cleaned and processed 'tokenized_word_w_stopwords' string,
# which likely contains the cleaned words from the 'comment_words' string after tokenization, filtering, and lemmatization
print(tokenized_word_w_stopwords)


Series([], )
series


In [18]:
# The cleaned and processed words will be stored in a new column named 'tokenized_word_w_stopwords' in the DataFrame
df['tokenized_word_w_stopwords'] = df['Content'].apply(lambda x: clean_contents_w_stopwords(x))
df 

,ID,Category,Content,tokenized_word_w_stopwords
0,15603,Entertainment,"\n Excerpts from the Clipper announcement, wit...",excerpt clipper announcement question ability ...
1,15617,Entertainment,"\n In article <strnlghtC5toC6.KIu@netcom.com>,...",article strnlghtctockiu netcomcom strnlght net...
2,15171,Entertainment,Sender: news@cujo.curtin.edu.au (News Manager)...,sender news cujocurtineduau news manager organ...
3,15818,Entertainment,\n I've been thinking about how difficult i...,thinking difficult would make available form e...
4,15824,Entertainment,\n In article merlin@lerami.lerctr.org (David ...,article merlin leramilerctrorg david hayes wri...
...,...,...,...,...
9135,15815,Crime,Distribution: world\n Message-ID: <1r7p86$82@a...,distribution world messageid accessdigexnet re...
9136,15801,Crime,Nntp-Posting-Host: mycroft.rand.org\n \n I was...,nntppostinghost mycroftrandorg annoyed find po...
9137,15632,Crime,\n In article <1993Apr21.204036.13723@rick.dgb...,article rickdgbtdocca jhan debradgbtdocca jerr...
9138,15626,Crime,\n strnlght@netcom.com (David Sternlight) writ...,strnlght netcomcom david sternlight writes not...


In [19]:
# Extract the 'tokenized_word_w_stopwords' column values from the DataFrame 'df' for each category
# and store them in separate variables for each category


politics_content = df[df['Category'] == 'Politics']['tokenized_word_w_stopwords'].values
politics_content = " ".join(politics_content)

science_content = df[df['Category'] == 'Science']['tokenized_word_w_stopwords'].values
science_content = " ".join(science_content)

# Store the cleaned and processed contents for each category in a list for further processing
contents = [politics_content, science_content]


In [20]:
df.drop_duplicates(subset=['ID', 'Content'], inplace=True, keep=False)

In [21]:
df = df[df['Category'] != '.ipynb_checkpoints']

In [22]:
df['Category'].value_counts().plot.bar()

In [23]:
minimum_count = df.groupby(by=['Category']).count()['Content'].min()


In [24]:
# filtering as Politics and Science data is more comparitivly
politics_content = df[df['Category'] == 'Politics']['tokenized_word_w_stopwords'].values
politics_content = " ".join(politics_content)

science_content = df[df['Category'] == 'Science']['tokenized_word_w_stopwords'].values
science_content = " ".join(science_content)

contents = [politics_content, science_content]

In [27]:
def gen_wordcloud(content):
    wordcloud = WordCloud(width = 800, height = 800,
                    background_color ='white',
                    min_font_size = 10).generate(content)
    return wordcloud

fig, axs = plt.subplots(1, 2, figsize= (32,16), facecolor= None)

politics_wordcloud = gen_wordcloud(politics_content)
axs[0].imshow(politics_wordcloud)
axs[0].set_title('politics_wordcloud', fontweight="bold", size=30)
axs[0].axis("off")

science_wordcloud = gen_wordcloud(science_content)
axs[1].imshow(science_wordcloud)
axs[1].set_title('science_wordcloud', fontweight="bold", size=30)
axs[1].axis("off")

# plt.axis("off")
# plt.tight_layout(pad = 0)


(-0.5, 799.5, 799.5, -0.5)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.svm import SVC

In [29]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict, train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, classification_report, roc_curve, precision_recall_fscore_support, confusion_matrix, pairwise_distances
from sklearn.ensemble import StackingClassifier, VotingClassifier
#!pip install --upgrade scipy
from scipy.spatial.distance import squareform
import pickle

In [30]:
le = LabelEncoder()  # Create an instance of LabelEncoder
df['Classification'] = le.fit_transform(df['Category'])  # Encode the 'Category' column and store the encoded labels in a new column 'Classification'

"""
Mapping of original categories to encoded labels:
Crime : 0 
Entertainment : 1 
Politics : 2
Science : 3
"""

df = df.sample(frac=1)  # Shuffle the rows of the DataFrame randomly

# Create separate DataFrames for each category with a limited number of rows equal to 'minimum_count'
politics_df = df.loc[df['Category'] == 'Politics'][:minimum_count]
crime_df = df.loc[df['Category'] == 'Crime'][:minimum_count]
entertainment_df = df.loc[df['Category'] == 'Entertainment'][:minimum_count]
science_df = df.loc[df['Category'] == 'Science'][:minimum_count]

# Concatenate the separate DataFrames into a single DataFrame 'normal_dist_df'
normal_dist_df = pd.concat([politics_df, crime_df, entertainment_df, science_df])

tfidf = TfidfVectorizer(lowercase=False)  # Create an instance of TfidfVectorizer with 'lowercase' parameter set to False
train_vec = tfidf.fit_transform(normal_dist_df['tokenized_word_w_stopwords'])  # Apply TF-IDF vectorization on the 'tokenized_word_w_stopwords' column of 'normal_dist_df'
print(train_vec.shape)  # Print the shape of the resulting TF-IDF matrix 'train_vec'


/var/folders/wr/prwm0mbx7m98xhz8f24wmh5r0000gn/T/ipykernel_44705/2124524440.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(5746, 51700)


In [31]:
X_train, X_test, y_train, y_test = train_test_split(train_vec, normal_dist_df['Classification'], stratify=normal_dist_df['Classification'], test_size=0.2)

print("x_train", X_train.shape, type(X_train))
print("X_test", X_test.shape, type(X_test))
print("y_train", y_train.shape, type(y_train))
print("y_test", y_test.shape, type(y_test))

x_train (4596, 51700) <class 'scipy.sparse._csr.csr_matrix'>
X_test (1150, 51700) <class 'scipy.sparse._csr.csr_matrix'>
y_train (4596,) <class 'pandas.core.series.Series'>
y_test (1150,) <class 'pandas.core.series.Series'>


In [34]:
# Creating and training the SVC classifier
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Predicting on the test data
y_pred = clf.predict(X_test)

print("Support Vector Classifier")
print(classification_report(y_test, y_pred, target_names = ['Science', 'Politics']))
print("-"*20)

Support Vector Classifier
              precision    recall  f1-score   support

     Science       0.99      0.98      0.99       575
    Politics       0.98      0.99      0.99       575

    accuracy                           0.99      1150
   macro avg       0.99      0.99      0.99      1150
weighted avg       0.99      0.99      0.99      1150

--------------------
